In [1]:
# workaround for incredibly stupid vscode on snap bug
from os import environ
krb5ccname = environ["KRB5CCNAME"]
if "hostfs" in krb5ccname:
    krb5ccname = krb5ccname.replace("/var/lib/snapd/hostfs", "")
    environ["KRB5CCNAME"] = krb5ccname

In [2]:
import ROOT
from analysis_framework import Dataset
from ObjectSelectionHelper import ObjectSelectionHelper, make_lvec_M, make_lvec_E

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x861e0e0


In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
prod = False
# prod = True
no_rvec = True
# write_outputs = False
write_outputs = True
# plot_dir_postfix = "-new-cuts"
dataset_path = "data/datasets/pre-selected/checked-test.json"
output_path = "data/selected-objects"
output_meta_path = "data/datasets/selected-objects"
output_meta = f"{output_meta_path}/test.json"
checked_output_meta = f"{output_meta_path}/checked-test.json"
# output_collections = [
#     "true_lep_lvec", "true_nu_lvec", "true_quark1_lvec", "true_quark2_lvec",
#     "iso_lep_lvec", "nu_lvec", "R2Jet_sel1_lvec", "R2Jet_sel2_lvec",
#     ]
# true lvecs do not exist in every df so cannot be explicitly requested...
# urgh but empty snapshots are also not allowed
output_collections = r"(true_\w+_lvec)|(iso_lep_lvec)|(nu_lvec)|(R2Jet_sel1_lvec)|(R2Jet_sel2_lvec)"
# plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/min-higgs.json"
    dataset_path = "data/datasets/pre-selected/checked-signal-only.json"
    # output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/min-higgs-d"
    # output_meta_path = "data/datasets/pre-selected"
    # output_meta = f"{output_meta_path}/min-higgs-d.json"
    # checked_output_meta = f"{output_meta_path}/checked-min-higgs-d.json"
    # output_collections = ["PandoraPFOs", "MCParticlesSkimmed", "Refined2Jets", "PFO_ovl_idx", "PFO_not_ovl_idx"]
    # plot_dir = "plots/pre-selection/full"
    # plot_dir = f"plots/pre-selection/min-higgs{plot_dir_postfix}"


In [4]:
ROOT.EnableImplicitMT(n_threads)

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = ObjectSelectionHelper(dataset)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xc5fe920


In [7]:
analysis.init_categories()
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]

True


In [8]:
ROOT.gStyle.SetOptStat(1)

In [9]:
# import json
# print(json.dumps(analysis._categories, indent=2))

In [10]:
analysis.define_truth_objects(signal_category)

In [11]:
# make nominal iso lep
# try to make iso lep + brems, try to make cheated brems
analysis.Define("iso_lep_idx", "IsolatedElectrons_objIdx.index[0]")
analysis.Define("iso_lep_charge", "PandoraPFOs.charge[iso_lep_idx]")
analysis.Define("iso_lep_lvec", "ROOT::Math::PxPyPzEVector(PandoraPFOs.momentum.x[iso_lep_idx], PandoraPFOs.momentum.y[iso_lep_idx], PandoraPFOs.momentum.z[iso_lep_idx], PandoraPFOs.energy[iso_lep_idx])")
analysis.Define("iso_lep_E", "iso_lep_lvec.energy()")

analysis.Define("n_iso_gammas", "IsolatedPhotons_objIdx.index.size()")
analysis.Define("iso_gamma_idx", "IsolatedPhotons_objIdx.index[0]")
analysis.Define("iso_gamma_lvec", "ROOT::Math::PxPyPzEVector(PandoraPFOs.momentum.x[iso_gamma_idx], PandoraPFOs.momentum.y[iso_gamma_idx], PandoraPFOs.momentum.z[iso_gamma_idx], PandoraPFOs.energy[iso_gamma_idx])")

# analysis.Define("iso_lep_gamma_deltaR", "ROOT::Math::VectorUtil::DeltaR(iso_lep_lvec, iso_gamma_lvec)")
analysis.Define("iso_lep_gamma_angle", "ROOT::Math::VectorUtil::Angle(iso_lep_lvec, iso_gamma_lvec)")
analysis.Define("iso_lep_gamma_close", "iso_lep_gamma_angle > 0. && iso_lep_gamma_angle < 0.01")

analysis.Define("iso_lep_gamma_lvec", "n_iso_gammas > 0 && iso_lep_gamma_close ? iso_lep_lvec + iso_gamma_lvec : iso_lep_lvec")

In [12]:
analysis.define_only_on(signal_category, "true_lep_E", "true_lep_lvec.energy()")

analysis.define_deltas("iso_lep", "iso_lep_lvec", "true_lep_lvec", signal_category)
analysis.define_deltas("iso_lep_gamma", "iso_lep_gamma_lvec", "true_lep_lvec", signal_category)

analysis.define_only_on(signal_category, "iso_lep_delta_phi_q", "iso_lep_delta_phi * true_iso_lep_charge")
analysis.define_only_on(signal_category, "iso_lep_gamma_delta_phi_q", "iso_lep_gamma_delta_phi * true_iso_lep_charge")


In [13]:
ROOT.gInterpreter.Declare("#include \"analyzers.h\"")
# make nominal jets, clean jets, try to make cheated super clean jets
analysis.Define("R2Jet_lvecs", "return Map(Refined2Jets, [] (const auto& el) {return ROOT::Math::PxPyPzEVector(el.momentum.x, el.momentum.y, el.momentum.z, el.energy);})")
# FIXME: _Refined2Jets_particles column is missing!!!
analysis.Define("clean_R2Jets", "remove_constituents(Refined2Jets, _Refined2Jets_particles, PandoraPFOs, PFO_ovl_idx)")
analysis.Define("clean_R2Jet_lvecs", "return Map(clean_R2Jets, [] (const auto& el) {return ROOT::Math::PxPyPzEVector(el.momentum.x, el.momentum.y, el.momentum.z, el.energy);})")

analysis.Define("R2Jet1_lvec", "R2Jet_lvecs[0]")
analysis.Define("R2Jet2_lvec", "R2Jet_lvecs[1]")
analysis.Define("clean_R2Jet1_lvec", "clean_R2Jet_lvecs[0]")
analysis.Define("clean_R2Jet2_lvec", "clean_R2Jet_lvecs[1]")

# also define 3d angular distance between jet and q
analysis.define_only_on(signal_category, "R2Jet1_q1_angle", "ROOT::Math::VectorUtil::Angle(R2Jet1_lvec, true_quark1_lvec)")
analysis.define_only_on(signal_category, "R2Jet1_q2_angle", "ROOT::Math::VectorUtil::Angle(R2Jet1_lvec, true_quark2_lvec)")
analysis.define_only_on(signal_category, "R2Jet2_q1_angle", "ROOT::Math::VectorUtil::Angle(R2Jet2_lvec, true_quark1_lvec)")
analysis.define_only_on(signal_category, "R2Jet2_q2_angle", "ROOT::Math::VectorUtil::Angle(R2Jet2_lvec, true_quark2_lvec)")

# assign as correct the combination with smallest total angular distance
analysis.define_only_on(signal_category, "R2Jet_cmb1_scr", "R2Jet1_q1_angle * R2Jet2_q2_angle")
analysis.define_only_on(signal_category, "R2Jet_cmb2_scr", "R2Jet1_q2_angle * R2Jet2_q1_angle")

# smaller score == better
analysis.define_only_on(signal_category, "R2Jet_sel1_lvec", "R2Jet_cmb1_scr < R2Jet_cmb2_scr ? R2Jet1_lvec : R2Jet2_lvec")
analysis.define_only_on(signal_category, "R2Jet_sel2_lvec", "R2Jet_cmb1_scr < R2Jet_cmb2_scr ? R2Jet2_lvec : R2Jet1_lvec")

# use the same combination for the clean jets
analysis.define_only_on(signal_category, "clean_R2Jet_sel1_lvec", "R2Jet_cmb1_scr < R2Jet_cmb2_scr ? clean_R2Jet1_lvec : clean_R2Jet2_lvec")
analysis.define_only_on(signal_category, "clean_R2Jet_sel2_lvec", "R2Jet_cmb1_scr < R2Jet_cmb2_scr ? clean_R2Jet2_lvec : clean_R2Jet1_lvec")

# define deltas for better combination
analysis.define_deltas("R2Jet_sel1", "R2Jet_sel1_lvec", "true_quark1_lvec", signal_category)
analysis.define_deltas("R2Jet_sel2", "R2Jet_sel2_lvec", "true_quark2_lvec", signal_category)

analysis.define_deltas("clean_R2Jet_sel1", "clean_R2Jet_sel1_lvec", "true_quark1_lvec", signal_category)
analysis.define_deltas("clean_R2Jet_sel2", "clean_R2Jet_sel2_lvec", "true_quark2_lvec", signal_category)

In [14]:
analysis.Define("nu_lvec", f"ROOT::Math::PxPyPzEVector(250*sin({x_angle/2}), 0., 0., 250.) - iso_lep_lvec - R2Jet1_lvec - R2Jet2_lvec")

analysis.define_deltas("nu", "nu_lvec", "true_nu_lvec", signal_category)

In [15]:
analysis.book_histogram_1D("true_lep_E", "true_lep_E", ("", "true lep E", 125, 0., 125.), categories=signal_category)

analysis.book_histogram_1D("iso_lep_delta_E", "iso_lep_delta_E", ("", ";iso lep #Delta E [GeV]", 150, -10., 5.), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_delta_E", "iso_lep_gamma_delta_E", ("", ";iso lep #Delta E [GeV]", 150, -10., 5.), categories=signal_category)

analysis.book_histogram_1D("iso_lep_delta_theta", "iso_lep_delta_theta", ("", ";iso lep #Delta #theta [rad]", 150, -0.0005, 0.0005), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_delta_theta", "iso_lep_gamma_delta_theta", ("", ";iso lep #Delta #theta [rad]", 150, -0.0005, 0.0005), categories=signal_category)

analysis.book_histogram_1D("iso_lep_delta_phi", "iso_lep_delta_phi", ("", ";iso lep #Delta #phi [rad]", 150, -0.0005, 0.0005), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_delta_phi", "iso_lep_gamma_delta_phi", ("", ";iso lep #Delta #phi [rad]", 150, -0.0005, 0.0005), categories=signal_category)

analysis.book_histogram_1D("iso_lep_delta_phi_q", "iso_lep_delta_phi_q", ("", ";iso lep #Delta #phi #dot q [rad]", 150, -0.0005, 0.0005), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_delta_phi_q", "iso_lep_gamma_delta_phi_q", ("", ";iso lep #Delta #phi #dot q [rad]", 150, -0.0005, 0.0005), categories=signal_category)

# analysis.book_histogram_1D("iso_lep_gamma_deltaR", "iso_lep_gamma_deltaR", ("", ";#Delta R lep gamma", 300, 0., 0.02), categories=signal_category)
analysis.book_histogram_1D("iso_lep_gamma_angle", "iso_lep_gamma_angle", ("", ";Angle lep gamma", 300, 0., 0.02), categories=signal_category)

In [16]:
analysis.book_histogram_1D("R2Jet_sel1_delta_E", "R2Jet_sel1_delta_E", ("", ";Jet1 #Delta E [GeV]", 150, -15., 15.), categories=signal_category)
analysis.book_histogram_1D("R2Jet_sel1_delta_theta", "R2Jet_sel1_delta_theta", ("", ";Jet1 #Delta #theta [rad]", 150, -0.1, 0.1), categories=signal_category)
analysis.book_histogram_1D("R2Jet_sel1_delta_phi", "R2Jet_sel1_delta_phi", ("", ";Jet1 #Delta #phi [rad]", 150, -0.1, 0.1), categories=signal_category)

analysis.book_histogram_1D("R2Jet_sel2_delta_E", "R2Jet_sel2_delta_E", ("", ";Jet2 #Delta E [GeV]", 150, -15., 15.), categories=signal_category)
analysis.book_histogram_1D("R2Jet_sel2_delta_theta", "R2Jet_sel2_delta_theta", ("", ";Jet2 #Delta #theta [rad]", 150, -0.1, 0.1), categories=signal_category)
analysis.book_histogram_1D("R2Jet_sel2_delta_phi", "R2Jet_sel2_delta_phi", ("", ";Jet2 #Delta #phi [rad]", 150, -0.1, 0.1), categories=signal_category)

analysis.book_histogram_1D("clean_R2Jet_sel1_delta_E", "clean_R2Jet_sel1_delta_E", ("", ";clean Jet1 #Delta E [GeV]", 150, -15., 15.), categories=signal_category)
analysis.book_histogram_1D("clean_R2Jet_sel1_delta_theta", "clean_R2Jet_sel1_delta_theta", ("", ";clean Jet1 #Delta #theta [rad]", 150, -0.1, 0.1), categories=signal_category)
analysis.book_histogram_1D("clean_R2Jet_sel1_delta_phi", "clean_R2Jet_sel1_delta_phi", ("", ";clean Jet1 #Delta #phi [rad]", 150, -0.1, 0.1), categories=signal_category)

analysis.book_histogram_1D("clean_R2Jet_sel2_delta_E", "clean_R2Jet_sel2_delta_E", ("", ";clean Jet2 #Delta E [GeV]", 150, -15., 15.), categories=signal_category)
analysis.book_histogram_1D("clean_R2Jet_sel2_delta_theta", "clean_R2Jet_sel2_delta_theta", ("", ";clean Jet2 #Delta #theta [rad]", 150, -0.1, 0.1), categories=signal_category)
analysis.book_histogram_1D("clean_R2Jet_sel2_delta_phi", "clean_R2Jet_sel2_delta_phi", ("", ";clean Jet2 #Delta #phi [rad]", 150, -0.1, 0.1), categories=signal_category)

In [17]:
analysis.book_histogram_1D("nu_delta_E", "nu_delta_E", ("", ";#nu #Delta E [GeV]", 150, -15., 15.), categories=signal_category)
analysis.book_histogram_1D("nu_delta_theta", "nu_delta_theta", ("", ";#nu #Delta #theta [rad]", 150, -0.1, 0.1), categories=signal_category)
analysis.book_histogram_1D("nu_delta_phi", "nu_delta_phi", ("", ";#nu #Delta #phi [rad]", 150, -0.1, 0.1), categories=signal_category)

In [18]:
analysis.book_histogram_1D("iso_lep_E", "iso_lep_E", ("", "iso lep E", 125, 0., 125.))

In [19]:
if write_outputs:
    analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [20]:
%%time
analysis.run()

CPU times: user 10.1 s, sys: 488 ms, total: 10.6 s
Wall time: 8.97 s


In [21]:
# if write_outputs:
    # analysis.check_snapshots("events", output_path, checked_output_meta)

In [22]:
analysis.draw_histogram("true_lep_E", categories=signal_category, draw_legend=False)

# analysis.draw_unscaled_histograms("iso_lep_gamma_deltaR", categories=signal_category)
# analysis.draw_unscaled_histograms("iso_lep_gamma_angle", categories=signal_category)
analysis.draw_unscaled_histograms("nu_delta_E", categories=signal_category)
analysis.draw_unscaled_histograms("nu_delta_theta", categories=signal_category)
analysis.draw_unscaled_histograms("nu_delta_phi", categories=signal_category)

In [23]:

# analysis.draw_unscaled_histograms("iso_lep_delta_E", categories=signal_category)
# analysis.draw_unscaled_histograms("iso_lep_gamma_delta_E", categories=signal_category)
analysis.compare_histograms_unscaled(["iso_lep_delta_E", "iso_lep_gamma_delta_E"], categories=signal_category)

# analysis.draw_histogram("iso_lep_delta_theta", categories=signal_category, draw_legend=False)
# analysis.draw_histogram("iso_lep_gamma_delta_theta", categories=signal_category, draw_legend=False)
analysis.compare_histograms_unscaled(["iso_lep_delta_theta", "iso_lep_gamma_delta_theta"], categories=signal_category)

# analysis.draw_histogram("iso_lep_delta_phi", categories=signal_category, draw_legend=False)
# analysis.draw_histogram("iso_lep_gamma_delta_phi", categories=signal_category, draw_legend=False)
analysis.compare_histograms_unscaled(["iso_lep_delta_phi", "iso_lep_gamma_delta_phi"], categories=signal_category)

# analysis.draw_histogram("iso_lep_delta_phi_q", categories=signal_category, draw_legend=False)
# analysis.draw_histogram("iso_lep_gamma_delta_phi_q", categories=signal_category, draw_legend=False)
analysis.compare_histograms_unscaled(["iso_lep_delta_phi_q", "iso_lep_gamma_delta_phi_q"], categories=signal_category)


In [24]:
analysis.compare_histograms_unscaled(["true_lep_E", "iso_lep_E"], categories=signal_category)

In [25]:
# analysis.draw_unscaled_histograms("R2Jet_sel1_delta_E", categories=signal_category)
# analysis.draw_unscaled_histograms("R2Jet_sel1_delta_theta", categories=signal_category)
# analysis.draw_unscaled_histograms("R2Jet_sel1_delta_phi", categories=signal_category)

# analysis.draw_unscaled_histograms("R2Jet_sel2_delta_E", categories=signal_category)
# analysis.draw_unscaled_histograms("R2Jet_sel2_delta_theta", categories=signal_category)
# analysis.draw_unscaled_histograms("R2Jet_sel2_delta_phi", categories=signal_category)

analysis.compare_histograms_unscaled(["R2Jet_sel1_delta_E", "clean_R2Jet_sel1_delta_E"], categories=signal_category)
analysis.compare_histograms_unscaled(["R2Jet_sel1_delta_theta", "clean_R2Jet_sel1_delta_theta"], categories=signal_category)
analysis.compare_histograms_unscaled(["R2Jet_sel1_delta_phi", "clean_R2Jet_sel1_delta_phi"], categories=signal_category)

analysis.compare_histograms_unscaled(["R2Jet_sel2_delta_E", "clean_R2Jet_sel2_delta_E"], categories=signal_category)
analysis.compare_histograms_unscaled(["R2Jet_sel2_delta_theta", "clean_R2Jet_sel2_delta_theta"], categories=signal_category)
analysis.compare_histograms_unscaled(["R2Jet_sel2_delta_phi", "clean_R2Jet_sel2_delta_phi"], categories=signal_category)